In [1]:
#reference :http://scikit-image.org/docs/dev/auto_examples/plot_hog.html

data_root = '/home/spuran/Caffe/caffe/data/Images/'

import os
import time

import numpy as np
import matplotlib.pyplot as plt

from skimage.feature import hog
from skimage.transform import resize
from skimage import io, data, color, exposure

In [2]:
import pandas as pd 
train_photos = pd.read_csv('/home/spuran/ML_FINAL_PROJECT/new_train11.csv')
train_folder = data_root+'train/'
train_images_path = [os.path.join(train_folder, str(x)+'.jpg') for x in train_photos['id']]  # get full filename

num_train = len(train_images_path)
#some images which had error have been deleted along with their corresponding lables 
#these images were identiifed while converting them to lmdb format itself for my another trial

In [3]:
print "Number of training images: ", num_train


Number of training images:  38369


In [4]:
list_features =[]

for i in range(0, num_train):
    
    image = io.imread(train_images_path[i])
    image = color.rgb2gray(image)
    image_resized = resize(image, (224, 224))
    feature = hog(image_resized, orientations=8,
        pixels_per_cell=(16, 16), cells_per_block=(1, 1))
    list_features.append(feature)
    

In [5]:
print len(list_features)

38369


In [6]:
feature_array = np.asarray(list_features)
X_train = feature_array

In [7]:
print feature_array.shape

(38369, 1568)


In [8]:
import numpy as np


list1 =[]
i =0
path = '/home/spuran/ML_FINAL_PROJECT/new_train11.csv'
with open(path,'r') as csvfile:
    data = csvfile.readlines()
    for line in data:
        array = line.split(",")
        if array[1] != "label":
            if i == 0:
                junk = array[1]

            else:
                str_label= array[1]
                str_label = str_label[1:-3]
                int_label = int(str_label)
            # print str_label
            # print type(array[1])
                list1.append([(int_label)])
        i+=1


labels = np.asarray(list1)
print len(labels)
print labels[9087]

38369
[8]


In [9]:
y_train = labels

In [10]:
import pandas as pd


test_photos = pd.read_csv(data_root+'test.csv',delim_whitespace=True)
test_folder = data_root+'test/'
test_images = [os.path.join(test_folder, str(x)+'.jpg') for x in test_photos['id']]

num_test = len(test_images)
print "Number of test images: ", num_test


Number of test images:  19646


In [11]:
list_features_test = []
error_list =[]
for i in range(0, num_test):
    try: 
        image = io.imread(test_images[i])
        image = color.rgb2gray(image)
        image_resized = resize(image, (224, 224))
        feature = hog(image_resized, orientations=8,
            pixels_per_cell=(16, 16), cells_per_block=(1, 1))
        list_features_test.append(feature)
    except(NameError,ValueError,IOError,MemoryError):
        print "there is an error"
        error_list.append(test_images[i])
        continue

In [12]:
feature_array_test = np.asarray(list_features_test)
X_test = feature_array_test

In [13]:
print feature_array_test.shape

(19646, 1568)


In [60]:
from sklearn import svm, datasets
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
# from sklearn.preprocessing import MultiLabelBinarizer

import time
t=time.time()

 #Convert list of labels to binary matrix

random_state = np.random.RandomState(0)
X_ptrain, X_ptest, y_ptrain, y_ptest = train_test_split(X_train, y_train, test_size=.2,random_state=random_state)

print "Time passed: ", "{0:.1f}".format(time.time()-t), "sec"

Time passed:  0.2 sec


In [ ]:
from sklearn.linear_model import LogisticRegression
OVR = OneVsRestClassifier(LogisticRegression())#using logistic regression for classsifciation
OVR.fit(X_ptrain, y_ptrain)

In [46]:
from sklearn.metrics import log_loss

In [44]:
print "accuracy", OVR.score(X_ptest,y_ptest)

accurayc 0.424420119885


In [48]:
Y1_final =OVR.predict_proba(X_ptest)
print (Y1_final[0])

[ 0.27950595  0.10718961  0.10542525  0.05972766  0.13607509  0.2463032
  0.00486557  0.06090766]


In [49]:
print log_loss(y_ptest,Y1_final)

1.56293734709


In [121]:
test_ids = [[str(x)] for x in test_photos['id']]

In [122]:
print type(test_ids)
a = np.asarray(test_ids)
print type(a),a.shape,a[1]

<type 'list'>
<type 'numpy.ndarray'> (19646, 1) ['53290']


In [127]:
import csv



# with open(data_root+"submission_HOG.csv",'w') as f:
#         wr = csv.writer(f, quoting=csv.QUOTE_ALL)
#         wr.writerow(["label","col1","col2","col3","col4","col5","col6","col7","col8"])
a1 = np.hstack((a,Y_final))
np.savetxt("submisoon1.csv",Y_final.astype(dtype = float),delimiter =",",fmt = '%1.5f')
# np.savetxt("submisoon1.csv",a1.astype(dtype = float),delimiter =",",fmt = '%1.5f')
#         for a in a1 :
#             wr.writerow(a)
#         print a1[0]
        
print error_list
#             wr.writerow(a2)
         
    # f = open('train11.txt','w')
    

['/home/spuran/Caffe/caffe/data/Images/test/79616.jpg', '/home/spuran/Caffe/caffe/data/Images/test/76787.jpg']
